In [ ]:
!pip install sqlite3

In [ ]:
def parse_line(line):
    error_cls = False
    warning_cls = False
    perm_cls =False
    ssh_cls = False
    other_cls = True
    # other_cls = False
    line = line.lower()
    # print(line)
    ts,rsc = line[:16].strip(),line[16:].split(':')[0]
    msg_idx = line[16:].index(':')
    msg = line[16:][msg_idx+1:].strip()
    if('error' in msg):
      error_cls = True
      other_cls =False
    if('warning' in msg):
      warning_cls = True
      other_clas = False
    # if('permission denied' in msg):
    #   perm_cls = True
    if('ssh' in msg):
      ssh_cls = True
      other_cls = False
    
    return ts,rsc,msg,error_cls,warning_cls,ssh_cls,other_cls

In [ ]:
import sqlite3
from datetime import datetime
import re

# Connect to SQLite database (it will be created if it doesn't exist)
conn = sqlite3.connect('logs.db')
cursor = conn.cursor()

# Create the table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS logs (
        timestamp TEXT,
        resource TEXT,
        message TEXT,
        error_cls BOOLEAN,
        warning_cls BOOLEAN,
        ssh_cls BOOLEAN,
        other_cls BOOLEAN
    )
''')

# Function to parse a log line
def parse_log_line(line):
    match = re.match(r'(\w+\s\d+\s\d+:\d+:\d+)\s([\w\-]+)\s([\w\-]+):\s(.+)', line)
    if match:
        date_str, _, source, message = match.groups()
        date = datetime.strptime(date_str, '%b %d %H:%M:%S').date()
        time = datetime.strptime(date_str, '%b %d %H:%M:%S').time()
        return str(date), str(time), source, message
    return None

# Read and parse the log file
line_ctr = 0
with open('/content/loganalysis/data/final_log.out', 'r') as file:
    for line in file:
      if(not line[:3] == 'Nov'):
        continue
      parsed_line = parse_line(line)
      if parsed_line:
        line_ctr+=1
        # print(parsed_line)
        cursor.execute('INSERT INTO logs (timestamp, resource, message, error_cls,warning_cls,ssh_cls,other_cls) VALUES (?, ?, ?, ?, ?, ?, ?)', parsed_line)
print(line_ctr)
conn.commit()
conn.close()

In [ ]:
# Connect to the logs.db file
connection = sqlite3.connect("logs.db")

# Create a cursor object
cursor = connection.cursor()

# Execute a query to get the names of all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch the result as a list of tuples
tables = cursor.fetchall()

# Print the names of the tables
print("The tables in the database are:")
for table in tables:
    print(table[0])

# Choose a table to read (for example, the first one)
table_name = tables[0][0]

# Execute a query to get the names of all columns in the table
cursor.execute(f"PRAGMA table_info({table_name});")

# Fetch the result as a list of tuples
columns = cursor.fetchall()

# Print the names of the columns
print(f"The columns in the table {table_name} are:")
for column in columns:
    print(column[1])


cursor.execute("SELECT COUNT(*) FROM logs;")
print(cursor.fetchall())
# Close the connection
connection.close()

In [ ]:
# Connect to the logs.db file
connection = sqlite3.connect("logs.db")

# Create a cursor object
cursor = connection.cursor()

# Execute a query to get the names of all tables in the database
cursor.execute("SELECT * FROM logs LIMIT 5;")

# Fetch the result as a list of tuples
output = cursor.fetchall()
print(output)

# Close the connection
connection.close()

In [ ]:
# Connect to the logs.db file
connection = sqlite3.connect("logs.db")

# Create a cursor object
cursor = connection.cursor()

# Execute a query to get the names of all tables in the database
cursor.execute("SELECT SUM(ssh_cls) FROM logs ;")

# Fetch the result as a list of tuples
output = cursor.fetchall()
print(output)

# Close the connection
connection.close()